# 태양광 발전량 예측

이 노트북에서 다루는 내용:
1. **탐색적 데이터 분석 (EDA)**
2. **데이터 시각화**
3. **데이터 전처리**
4. **특성 공학**
5. **분위수 회귀 모델링**
6. **제출 파일 생성**

## 1. 라이브러리 임포트

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import os
import warnings
import matplotlib.font_manager as fm

warnings.filterwarnings('ignore')

# 스타일 먼저 설정 (폰트보다 먼저!)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Windows 한글 폰트 직접 경로 지정
font_path = 'C:/Windows/Fonts/malgun.ttf'

# 폰트 등록 및 설정
fm.fontManager.addfont(font_path)
font_prop = fm.FontProperties(fname=font_path)
font_name = font_prop.get_name()

# rc 설정으로 전역 폰트 변경
plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

print(f'한글 폰트 설정 완료: {font_name}')
print('라이브러리 임포트 완료!')

In [ ]:
# 한글 폰트 테스트
print(f'현재 폰트: {plt.rcParams["font.family"]}')

fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(['월요일', '화요일', '수요일', '목요일', '금요일'], [10, 25, 15, 30, 20], color='skyblue', edgecolor='navy')
ax.set_xlabel('요일')
ax.set_ylabel('발전량 (kW)')
ax.set_title('한글 폰트 테스트 - 태양광 발전량')
plt.tight_layout()
plt.show()

print('\n한글이 보이면 설정 완료!')

## 2. 데이터 로드

In [ ]:
# 학습 데이터 로드
train = pd.read_csv('data/train/train.csv')
print(f'학습 데이터 크기: {train.shape}')

# 테스트 파일 로드
test_files = sorted(glob('data/test/*.csv'), key=lambda x: int(os.path.basename(x).split('.')[0]))
print(f'테스트 파일 수: {len(test_files)}')

# 샘플 제출 파일 로드
sample_submission = pd.read_csv('data/sample_submission.csv')
print(f'샘플 제출 파일 크기: {sample_submission.shape}')

In [ ]:
# 학습 데이터 확인
train.head(10)

In [ ]:
# 샘플 제출 파일 확인
sample_submission.head(10)

## 3. 탐색적 데이터 분석 (EDA)

### 3.1 기본 정보

In [ ]:
# 기본 정보
print('='*60)
print('데이터 정보')
print('='*60)
train.info()

In [ ]:
# 통계 요약
train.describe()

In [ ]:
# 결측치 확인
print('='*60)
print('결측치 현황')
print('='*60)
missing = train.isnull().sum()
missing_pct = (train.isnull().sum() / len(train)) * 100
missing_df = pd.DataFrame({'결측치 수': missing, '결측치 비율(%)': missing_pct})
print(missing_df)

In [ ]:
# 컬럼별 고유값 수
print('='*60)
print('고유값 수')
print('='*60)
for col in train.columns:
    print(f'{col}: {train[col].nunique()}개 고유값')

### 3.2 타겟 변수 분석

In [ ]:
# 타겟 분포
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 히스토그램
axes[0].hist(train['TARGET'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('TARGET (발전량)')
axes[0].set_ylabel('빈도')
axes[0].set_title('TARGET 분포')

# 히스토그램 (0 제외)
axes[1].hist(train[train['TARGET'] > 0]['TARGET'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_xlabel('TARGET (발전량)')
axes[1].set_ylabel('빈도')
axes[1].set_title('TARGET 분포 (0 제외)')

# 박스 플롯
axes[2].boxplot(train['TARGET'])
axes[2].set_ylabel('TARGET')
axes[2].set_title('TARGET 박스 플롯')

plt.tight_layout()
plt.show()

# 0값 비율
zero_pct = (train['TARGET'] == 0).sum() / len(train) * 100
print(f'\nTARGET이 0인 비율: {zero_pct:.2f}%')

### 3.3 시간대별 분석

In [ ]:
# 시간대별 평균 TARGET
hourly_avg = train.groupby('Hour')['TARGET'].mean()

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# 막대 그래프
axes[0].bar(hourly_avg.index, hourly_avg.values, color='skyblue', edgecolor='navy')
axes[0].set_xlabel('시간')
axes[0].set_ylabel('평균 TARGET')
axes[0].set_title('시간대별 평균 발전량')
axes[0].set_xticks(range(0, 24))

# 시간-분 히트맵
pivot = train.pivot_table(values='TARGET', index='Hour', columns='Minute', aggfunc='mean')
sns.heatmap(pivot, cmap='YlOrRd', ax=axes[1], annot=True, fmt='.1f')
axes[1].set_title('시간/분별 평균 발전량')

plt.tight_layout()
plt.show()

In [ ]:
# 일별 패턴 샘플 (처음 7일)
sample_days = train[train['Day'] < 7].copy()
sample_days['time_idx'] = sample_days['Day'] * 48 + sample_days['Hour'] * 2 + sample_days['Minute'] // 30

plt.figure(figsize=(16, 6))
plt.plot(sample_days['time_idx'], sample_days['TARGET'], linewidth=1)
plt.xlabel('시간 인덱스 (30분 간격)')
plt.ylabel('발전량 (kW)')
plt.title('발전량 패턴 (처음 7일)')
plt.grid(True, alpha=0.3)
plt.show()

### 3.4 특성 분석

In [ ]:
# 특성 분포
features = ['DHI', 'DNI', 'WS', 'RH', 'T']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, feat in enumerate(features):
    axes[i].hist(train[feat], bins=50, edgecolor='black', alpha=0.7)
    axes[i].set_xlabel(feat)
    axes[i].set_ylabel('빈도')
    axes[i].set_title(f'{feat} 분포')

axes[-1].axis('off')  # 마지막 빈 서브플롯 숨기기
plt.tight_layout()
plt.show()

In [ ]:
# 상관관계 행렬
plt.figure(figsize=(10, 8))
corr_matrix = train[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.3f',
            linewidths=0.5, square=True)
plt.title('상관관계 행렬')
plt.tight_layout()
plt.show()

In [ ]:
# TARGET과의 산점도
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, feat in enumerate(features):
    # 오버플로팅 방지를 위한 샘플링
    sample = train.sample(min(5000, len(train)), random_state=42)
    axes[i].scatter(sample[feat], sample['TARGET'], alpha=0.3, s=10)
    axes[i].set_xlabel(feat)
    axes[i].set_ylabel('TARGET')
    axes[i].set_title(f'{feat} vs TARGET')

axes[-1].axis('off')
plt.tight_layout()
plt.show()

### 3.5 일사량 분석

In [ ]:
# 시간대별 DHI, DNI 패턴
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# 시간대별 DHI
dhi_hourly = train.groupby('Hour')['DHI'].mean()
axes[0].bar(dhi_hourly.index, dhi_hourly.values, color='orange', edgecolor='darkorange')
axes[0].set_xlabel('시간')
axes[0].set_ylabel('평균 DHI')
axes[0].set_title('시간대별 평균 DHI (산란일사량)')
axes[0].set_xticks(range(0, 24))

# 시간대별 DNI
dni_hourly = train.groupby('Hour')['DNI'].mean()
axes[1].bar(dni_hourly.index, dni_hourly.values, color='red', edgecolor='darkred')
axes[1].set_xlabel('시간')
axes[1].set_ylabel('평균 DNI')
axes[1].set_title('시간대별 평균 DNI (직달일사량)')
axes[1].set_xticks(range(0, 24))

plt.tight_layout()
plt.show()

In [ ]:
# GHI (전천일사량) 계산
# GHI = DHI + DNI * cos(천정각)
# 단순화를 위해 DHI + DNI를 근사값으로 사용
train['GHI_proxy'] = train['DHI'] + train['DNI'] * 0.5  # 단순화된 근사

plt.figure(figsize=(12, 5))
sample = train.sample(min(5000, len(train)), random_state=42)
plt.scatter(sample['GHI_proxy'], sample['TARGET'], alpha=0.3, s=10)
plt.xlabel('GHI 근사값 (DHI + 0.5*DNI)')
plt.ylabel('TARGET')
plt.title('GHI 근사값 vs 발전량')
plt.show()

## 4. 데이터 전처리

In [ ]:
def load_test_data(test_files):
    """모든 테스트 파일을 로드하여 결합"""
    test_data = []
    for file in test_files:
        file_id = int(os.path.basename(file).split('.')[0])
        df = pd.read_csv(file)
        df['file_id'] = file_id
        test_data.append(df)
    return test_data

test_data = load_test_data(test_files)
print(f'{len(test_data)}개의 테스트 파일 로드 완료')
print(f'샘플 테스트 파일 크기: {test_data[0].shape}')

In [ ]:
# 테스트 데이터 구조 확인
test_data[0].head()

In [ ]:
# 테스트 데이터의 Day 확인
print('테스트 파일 0의 Day:', test_data[0]['Day'].unique())
print('\nDay 7과 Day 8을 예측해야 함 (일당 48행 = 파일당 96행)')

## 5. 특성 공학

In [ ]:
def create_features(df, is_train=True):
    """모델링을 위한 특성 생성"""
    df = df.copy()
    
    # 시간 특성
    df['time_idx'] = df['Hour'] * 2 + df['Minute'] // 30  # 0-47
    
    # 순환 시간 특성
    df['hour_sin'] = np.sin(2 * np.pi * df['Hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['Hour'] / 24)
    df['time_sin'] = np.sin(2 * np.pi * df['time_idx'] / 48)
    df['time_cos'] = np.cos(2 * np.pi * df['time_idx'] / 48)
    
    # 주간 시간대 지표 (근사)
    df['is_daylight'] = ((df['Hour'] >= 6) & (df['Hour'] <= 18)).astype(int)
    
    # 일사량 특성
    df['GHI_proxy'] = df['DHI'] + df['DNI'] * 0.5
    df['solar_ratio'] = df['DHI'] / (df['DNI'] + 1)  # 0으로 나누기 방지
    df['total_irradiance'] = df['DHI'] + df['DNI']
    
    # 기상 상호작용
    df['T_RH'] = df['T'] * df['RH']
    df['WS_T'] = df['WS'] * df['T']
    df['DHI_T'] = df['DHI'] * df['T']
    df['DNI_T'] = df['DNI'] * df['T']
    
    # 온도 효율 계수 (고온 = 낮은 효율)
    df['temp_efficiency'] = 1 - 0.005 * (df['T'] - 25).clip(lower=0)
    
    # 습도 계수
    df['humidity_factor'] = (100 - df['RH']) / 100
    
    return df

# 학습 데이터에 특성 공학 적용
train_fe = create_features(train, is_train=True)
print(f'특성 공학 후 학습 데이터 크기: {train_fe.shape}')
train_fe.head()

In [ ]:
def create_lag_features(df, group_col=None, target_col='TARGET', lags=[1, 2, 3]):
    """이전 시점 기반 지연 특성 생성"""
    df = df.copy()
    
    for lag in lags:
        if group_col:
            df[f'target_lag_{lag}'] = df.groupby(group_col)[target_col].shift(lag)
        else:
            df[f'target_lag_{lag}'] = df[target_col].shift(lag)
    
    return df

# 일별 패턴 롤링 통계 생성
def create_daily_stats(df):
    """일별 집계 통계 생성"""
    df = df.copy()
    
    # 모든 날의 시간대별 평균 (일반적인 일별 패턴 캡처)
    hour_stats = df.groupby(['Hour', 'Minute']).agg({
        'TARGET': ['mean', 'std', 'min', 'max'],
        'DHI': 'mean',
        'DNI': 'mean',
        'T': 'mean'
    }).reset_index()
    hour_stats.columns = ['Hour', 'Minute', 'target_hour_mean', 'target_hour_std', 
                          'target_hour_min', 'target_hour_max',
                          'DHI_hour_mean', 'DNI_hour_mean', 'T_hour_mean']
    
    df = df.merge(hour_stats, on=['Hour', 'Minute'], how='left')
    
    return df, hour_stats

train_fe, hour_stats = create_daily_stats(train_fe)
print(f'일별 통계 추가 후 학습 데이터 크기: {train_fe.shape}')

In [ ]:
# 생성된 새 특성 표시
new_features = [col for col in train_fe.columns if col not in train.columns]
print('생성된 새 특성:')
for feat in new_features:
    print(f'  - {feat}')

## 6. 분위수 회귀 모델링

In [ ]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import lightgbm as lgb

# 분위수 정의
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
# 특성 준비
feature_cols = ['Hour', 'Minute', 'DHI', 'DNI', 'WS', 'RH', 'T',
                'time_idx', 'hour_sin', 'hour_cos', 'time_sin', 'time_cos',
                'is_daylight', 'GHI_proxy', 'solar_ratio', 'total_irradiance',
                'T_RH', 'WS_T', 'DHI_T', 'DNI_T', 'temp_efficiency', 'humidity_factor',
                'target_hour_mean', 'target_hour_std', 'target_hour_min', 'target_hour_max',
                'DHI_hour_mean', 'DNI_hour_mean', 'T_hour_mean']

X = train_fe[feature_cols]
y = train_fe['TARGET']

print(f'특성 크기: {X.shape}')
print(f'타겟 크기: {y.shape}')

In [ ]:
# 시간 기반 학습/검증 분할
# 마지막 20% 일수를 검증용으로 사용
n_days = train_fe['Day'].max() + 1
val_start_day = int(n_days * 0.8)

train_mask = train_fe['Day'] < val_start_day
val_mask = train_fe['Day'] >= val_start_day

X_train, X_val = X[train_mask], X[val_mask]
y_train, y_val = y[train_mask], y[val_mask]

print(f'학습 세트: {X_train.shape}')
print(f'검증 세트: {X_val.shape}')

In [ ]:
def pinball_loss(y_true, y_pred, quantile):
    """특정 분위수에 대한 핀볼 손실 계산"""
    errors = y_true - y_pred
    return np.mean(np.maximum(quantile * errors, (quantile - 1) * errors))

def train_quantile_model(X_train, y_train, X_val, y_val, quantile, params=None):
    """특정 분위수에 대한 LightGBM 모델 학습"""
    if params is None:
        params = {
            'objective': 'quantile',
            'alpha': quantile,
            'n_estimators': 500,
            'learning_rate': 0.05,
            'max_depth': 8,
            'num_leaves': 50,
            'min_child_samples': 20,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42,
            'n_jobs': -1,
            'verbosity': -1
        }
    
    model = LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(50, verbose=False)]
    )
    
    return model

In [ ]:
# 모든 분위수에 대해 모델 학습
models = {}
val_predictions = {}

for q in quantiles:
    print(f'분위수 {q} 모델 학습 중...')
    model = train_quantile_model(X_train, y_train, X_val, y_val, q)
    models[q] = model
    
    # 검증 예측
    y_pred = model.predict(X_val)
    y_pred = np.clip(y_pred, 0, None)  # 음수 예측값 클리핑
    val_predictions[q] = y_pred
    
    loss = pinball_loss(y_val, y_pred, q)
    print(f'  분위수 {q} - 핀볼 손실: {loss:.4f}')

print('\n모든 모델 학습 완료!')

In [ ]:
# 평균 핀볼 손실 계산
total_loss = 0
for q in quantiles:
    total_loss += pinball_loss(y_val, val_predictions[q], q)
avg_loss = total_loss / len(quantiles)
print(f'평균 핀볼 손실: {avg_loss:.4f}')

### 6.1 특성 중요도

In [ ]:
# 중앙값 모델(q=0.5)의 특성 중요도
median_model = models[0.5]
importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': median_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 8))
plt.barh(importance['feature'][:20], importance['importance'][:20])
plt.xlabel('중요도')
plt.ylabel('특성')
plt.title('상위 20개 특성 중요도 (중앙값 모델)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

### 6.2 검증 예측 시각화

In [ ]:
# 검증 예측 플롯
sample_idx = np.arange(0, min(500, len(y_val)))

plt.figure(figsize=(16, 6))
plt.plot(sample_idx, y_val.iloc[sample_idx].values, 'b-', label='실제값', linewidth=2)
plt.fill_between(sample_idx, val_predictions[0.1][sample_idx], val_predictions[0.9][sample_idx], 
                 alpha=0.3, color='orange', label='10-90% 구간')
plt.fill_between(sample_idx, val_predictions[0.3][sample_idx], val_predictions[0.7][sample_idx], 
                 alpha=0.4, color='orange', label='30-70% 구간')
plt.plot(sample_idx, val_predictions[0.5][sample_idx], 'r--', label='중앙값 예측', linewidth=1)
plt.xlabel('샘플 인덱스')
plt.ylabel('발전량')
plt.title('분위수 구간을 포함한 검증 예측')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. 테스트 예측 및 제출 파일 생성

In [ ]:
# 전체 학습 데이터로 재학습
print('전체 학습 데이터로 모델 재학습 중...')

final_models = {}
for q in quantiles:
    print(f'분위수 {q} 모델 학습 중...')
    
    params = {
        'objective': 'quantile',
        'alpha': q,
        'n_estimators': 500,
        'learning_rate': 0.05,
        'max_depth': 8,
        'num_leaves': 50,
        'min_child_samples': 20,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': -1
    }
    
    model = LGBMRegressor(**params)
    model.fit(X, y)
    final_models[q] = model

print('\n모든 최종 모델 학습 완료!')

In [ ]:
def create_future_predictions(test_df, hour_stats, file_id, final_models, quantiles, feature_cols):
    """
    테스트 데이터(Day 0-6)를 기반으로 Day 7, Day 8의 예측을 생성
    
    Day 7, Day 8에 대한 기상 데이터가 없으므로:
    - 시간대별 통계(hour_stats)를 사용
    - 마지막 날(Day 6)의 기상 패턴을 참조
    """
    # 마지막 날(Day 6)의 데이터 가져오기
    last_day_data = test_df[test_df['Day'] == 6].copy()
    
    predictions_list = []
    
    # Day 7과 Day 8에 대해 예측 생성
    for pred_day in [7, 8]:
        # 각 시간대에 대해 특성 생성 (48개 시간대: 24시간 * 2)
        for hour in range(24):
            for minute in [0, 30]:
                # 마지막 날의 동일 시간대 데이터 참조
                ref_data = last_day_data[(last_day_data['Hour'] == hour) & 
                                         (last_day_data['Minute'] == minute)]
                
                if len(ref_data) > 0:
                    ref_row = ref_data.iloc[0]
                    # 마지막 날의 기상 데이터 사용
                    DHI = ref_row['DHI']
                    DNI = ref_row['DNI']
                    WS = ref_row['WS']
                    RH = ref_row['RH']
                    T = ref_row['T']
                else:
                    # 참조 데이터가 없으면 hour_stats 사용
                    hour_stat = hour_stats[(hour_stats['Hour'] == hour) & 
                                           (hour_stats['Minute'] == minute)]
                    if len(hour_stat) > 0:
                        DHI = hour_stat['DHI_hour_mean'].values[0]
                        DNI = hour_stat['DNI_hour_mean'].values[0]
                        T = hour_stat['T_hour_mean'].values[0]
                    else:
                        DHI, DNI, T = 0, 0, 10
                    WS, RH = 2.0, 50.0
                
                # 특성 생성
                time_idx = hour * 2 + minute // 30
                hour_sin = np.sin(2 * np.pi * hour / 24)
                hour_cos = np.cos(2 * np.pi * hour / 24)
                time_sin = np.sin(2 * np.pi * time_idx / 48)
                time_cos = np.cos(2 * np.pi * time_idx / 48)
                is_daylight = 1 if 6 <= hour <= 18 else 0
                GHI_proxy = DHI + DNI * 0.5
                solar_ratio = DHI / (DNI + 1)
                total_irradiance = DHI + DNI
                T_RH = T * RH
                WS_T = WS * T
                DHI_T = DHI * T
                DNI_T = DNI * T
                temp_efficiency = 1 - 0.005 * max(0, T - 25)
                humidity_factor = (100 - RH) / 100
                
                # hour_stats에서 시간대별 통계 가져오기
                hour_stat = hour_stats[(hour_stats['Hour'] == hour) & 
                                       (hour_stats['Minute'] == minute)]
                if len(hour_stat) > 0:
                    target_hour_mean = hour_stat['target_hour_mean'].values[0]
                    target_hour_std = hour_stat['target_hour_std'].values[0]
                    target_hour_min = hour_stat['target_hour_min'].values[0]
                    target_hour_max = hour_stat['target_hour_max'].values[0]
                    DHI_hour_mean = hour_stat['DHI_hour_mean'].values[0]
                    DNI_hour_mean = hour_stat['DNI_hour_mean'].values[0]
                    T_hour_mean = hour_stat['T_hour_mean'].values[0]
                else:
                    target_hour_mean, target_hour_std = 0, 0
                    target_hour_min, target_hour_max = 0, 0
                    DHI_hour_mean, DNI_hour_mean, T_hour_mean = 0, 0, 10
                
                # 특성 딕셔너리 생성
                features = {
                    'Hour': hour, 'Minute': minute, 'DHI': DHI, 'DNI': DNI,
                    'WS': WS, 'RH': RH, 'T': T, 'time_idx': time_idx,
                    'hour_sin': hour_sin, 'hour_cos': hour_cos,
                    'time_sin': time_sin, 'time_cos': time_cos,
                    'is_daylight': is_daylight, 'GHI_proxy': GHI_proxy,
                    'solar_ratio': solar_ratio, 'total_irradiance': total_irradiance,
                    'T_RH': T_RH, 'WS_T': WS_T, 'DHI_T': DHI_T, 'DNI_T': DNI_T,
                    'temp_efficiency': temp_efficiency, 'humidity_factor': humidity_factor,
                    'target_hour_mean': target_hour_mean, 'target_hour_std': target_hour_std,
                    'target_hour_min': target_hour_min, 'target_hour_max': target_hour_max,
                    'DHI_hour_mean': DHI_hour_mean, 'DNI_hour_mean': DNI_hour_mean,
                    'T_hour_mean': T_hour_mean
                }
                
                # 예측용 DataFrame 생성
                X_pred = pd.DataFrame([features])[feature_cols]
                
                # 각 분위수에 대한 예측
                pred_row = {'id': f"{file_id}.csv_Day{pred_day}_{hour}h{minute:02d}m"}
                for q in quantiles:
                    pred_val = final_models[q].predict(X_pred)[0]
                    pred_row[f'q_{q}'] = max(0, pred_val)  # 음수 방지
                
                predictions_list.append(pred_row)
    
    return pd.DataFrame(predictions_list)

# 모든 테스트 파일에 대한 예측 생성
print('테스트 파일에 대한 예측 생성 중...')
all_predictions = []

for i, test_df in enumerate(test_data):
    file_id = test_df['file_id'].iloc[0]
    
    # Day 7, Day 8 예측 생성
    pred_df = create_future_predictions(
        test_df, hour_stats, file_id, final_models, quantiles, feature_cols
    )
    all_predictions.append(pred_df)
    
    if (i + 1) % 20 == 0:
        print(f'{i+1}/{len(test_data)} 파일 처리 완료')

print(f'\n총 처리된 파일 수: {len(all_predictions)}')

In [ ]:
# 모든 예측 결합
submission = pd.concat(all_predictions, ignore_index=True)

print(f'제출 파일 크기: {submission.shape}')
print(f'예상 크기: {sample_submission.shape}')
print(f'\\n제출 파일 컬럼: {submission.columns.tolist()}')
submission.head(10)

In [ ]:
# 제출 형식 검증
print('제출 파일 컬럼:', submission.columns.tolist())
print('샘플 제출 파일 컬럼:', sample_submission.columns.tolist())

# ID 일치 확인
submission_ids = set(submission['id'])
sample_ids = set(sample_submission['id'])

missing_ids = sample_ids - submission_ids
extra_ids = submission_ids - sample_ids

print(f'\\n누락된 ID 수: {len(missing_ids)}')
print(f'추가된 ID 수: {len(extra_ids)}')

if len(missing_ids) > 0:
    print(f'\\n누락된 ID 예시: {list(missing_ids)[:5]}')

In [ ]:
# 샘플 제출 파일과 정렬
# 컬럼명을 샘플 제출 파일 형식에 맞게 변경
submission_renamed = submission.rename(columns={
    f'q_{q}': f'q_{q}' for q in quantiles
})

# sample_submission의 컬럼명 확인 후 맞추기
sample_cols = sample_submission.columns.tolist()
print(f'샘플 제출 파일 컬럼명: {sample_cols}')

# 컬럼명 매핑 (q_0.1 형식으로)
col_mapping = {'id': 'id'}
for q in quantiles:
    col_mapping[f'q_{q}'] = f'q_{q}'

# 최종 제출 파일 생성
final_submission = sample_submission[['id']].merge(submission, on='id', how='left')

# 누락값을 0으로 채우기
final_submission = final_submission.fillna(0)

print(f'\\n최종 제출 파일 크기: {final_submission.shape}')
print(f'컬럼: {final_submission.columns.tolist()}')
final_submission.head(10)

In [ ]:
# 제출 파일 저장
final_submission.to_csv('submission.csv', index=False)
print('제출 파일이 submission.csv로 저장되었습니다.')

## 8. 요약 및 개선 방향

### 요약
- 태양광 발전량 데이터에 대한 EDA 수행
- 시간 기반 및 기상 기반 특성 생성
- 9개 분위수에 대한 LightGBM 분위수 회귀 모델 학습
- 테스트 데이터에 대한 예측 생성

### 잠재적 개선 방향
1. **특성 공학**
   - 이전 날의 지연 특성 추가
   - 계절 지표 포함
   - 태양 위치 계산 (천정각)
   
2. **모델 개선**
   - 교차 검증을 통한 하이퍼파라미터 튜닝
   - XGBoost, CatBoost와의 앙상블
   - 시퀀스 모델링을 위한 신경망
   
3. **검증 전략**
   - K-fold 시계열 교차 검증
   - 워크 포워드 검증

In [ ]:
print('분석 완료!')